In [ ]:
# Importing the required libraries
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from datetime import datetime

#Initialize enttities
url='https://web.archive.org/web/20230908091635%20/https://en.wikipedia.org/wiki/List_of_largest_banks'
table_attributes =['Name','MC_USD_Billion']
db_name = 'Banks.db'
table_name = 'Largest_banks'
output_path = './Largest_banks_data.csv'
log_file = 'code_log.txt'
csv_path = './exchange_rate.csv'
sql_connection = sqlite3.connect(db_name)
query_1=f"SELECT * FROM {table_name}"
query_2 = f"SELECT AVG(MC_GBP_Billion) FROM {table_name}"
query_3 = f"SELECT Name from {table_name} LIMIT 5"
# Code for ETL operations on Country-GDP data

def log_progress(message):
    ''' This function logs the mentioned message at a given stage of the code execution to a log file. Function returns nothing'''
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open(log_file,"a") as f: 
        f.write(timestamp + ',' + message + '\n') 

def extract(url, table_attributes):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''
    # Initialize an empty DataFrame with specified column names
    df = pd.DataFrame(columns=table_attributes)
    
    # Fetch the HTML content of a webpage at the specified URL
    html_page = requests.get(url).text
    
    # Parse the fetched HTML using BeautifulSoup
    data = BeautifulSoup(html_page, 'html.parser')
    
    # Find all 'table' elements within the parsed HTML
    tables = data.find_all('table')
    
    # Extract all rows ('tr' elements) from the first table
    rows = tables[0].find_all('tr')

    # Iterate over all rows, starting from the second row (skipping the header)
    for row in rows[1:]:
        # Find all 'td' elements (table data) within the row
        col = row.find_all('td')
        
        # Create a dictionary with data for the DataFrame,
        # extracting text from the 2nd and 3rd columns of the table row
        data_dict = {
            table_attributes[0]: col[1].get_text(strip=True),
            table_attributes[1]: col[2].get_text(strip=True)
        }
        
        # Create a DataFrame from the dictionary with a single row
        df1 = pd.DataFrame(data_dict, index=[0])
        
        # Append the new data to the main DataFrame, resetting the index
        df = pd.concat([df, df1], ignore_index=True)

    return df

def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''
    # Convert exchange rate DataFrame to dictionary with the first column as keys and the second as values
    exchange_rate_df = pd.read_csv(csv_path)
    exchange_rate = exchange_rate_df.set_index('Currency')['Rate'].to_dict()
    
    #Convert MC_USD_Billion to float
    df['MC_USD_Billion']=df['MC_USD_Billion'].astype(float)
    
    #Create new columns for EUR, GBP and INR currencies
    df['MC_GBP_Billion'] = [np.round(x*exchange_rate['GBP'],2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x*exchange_rate['EUR'],2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x*exchange_rate['INR'],2) for x in df['MC_USD_Billion']]

    return df

def load_to_csv(df, output_path):
    df.to_csv(output_path)
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''

def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''

def run_query(query_statement, sql_connection):
    query_output = pd.read_sql(query_statement, sql_connection)
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''

''' Here, you define the required entities and call the relevant
functions in the correct order to complete the project. Note that this
portion is not inside any function.'''

# Log the initialization of the ETL process 
log_progress("Preliminaries complete. Initiating ETL process") 
 
# Log the beginning of the Extraction process 
log_progress("Data extraction complete. Initiating Transformation process") 
extracted_data = extract(url, table_attributes) 

# Log the beginning of the Transformation process 
log_progress("Data transformation complete. Initiating Loading process") 
transformed_data = transform(extracted_data, csv_path) 

# Log the beginning of the load to CSV process 
log_progress("Data saved to CSV file") 
load_to_csv(transformed_data, output_path) 

#Initiate sql connection
log_progress("SQL Connection initiated") 
sql_connection = sqlite3.connect(db_name)

#Log the beginning of the load to DB process 
log_progress("Data loaded to Database as a table, Executing queries") 
load_to_db(transformed_data, sql_connection, table_name)

#Log the beginning of the querying process 
log_progress("Process Complete") 
run_query(query_1,sql_connection)
run_query(query_2,sql_connection)
run_query(query_3,sql_connection)

#Closing connection
log_progress("Server Connection closed") 
sql_connection.close()